In [ ]:
import json
%run dataloader.ipynb
# %run dataloader_ori.ipynb

In [ ]:
import os
import random
import torch.nn.functional as F
import torch
import time
import datetime
import  torchvision
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import MultiStepLR


torch.cuda.is_available()
device = torch.device("cuda:0")
device
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
!nvidia-smi

In [ ]:
path1 = r'./Datasets/AiRound/aerial/'
train_images_path, train_images_label,val_images_path, val_images_label,test_images_path, test_images_label , num_classes=read_split_data(path1,0.7,0.1,0.2)


In [ ]:
# num_classes =9
model = torchvision.models.resnet50(pretrained = True)
model.fc = nn.Linear(2048,num_classes)
model.to(device)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)
scheduler = MultiStepLR(optimizer, milestones=[50,200], gamma=0.1)
ceriation = nn.CrossEntropyLoss().to(device)

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)

    
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def getTime():
    time_stamp = datetime.datetime.now()
    return time_stamp.strftime('%H:%M:%S')

In [11]:
def train(model, train_iter, ceriation, train_optimizer, num_prints=1):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':6.3f')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(
        len(train_iter),
        [batch_time, data_time, losses, top1], prefix="Train ")

    end = time.time()
    num_iter = int((len(train_iter)-1) / num_prints)
    for batch_idx in range(num_iter):
        try:
            images, labels,_ = train_iter.next()
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
        except StopIteration:
            break

        model.train()
        data_time.update(time.time() - end)
        train_optimizer.zero_grad()
        logits = model(images)
        loss = ceriation(logits, labels)
        loss.requires_grad_(True)
        loss.backward()
        train_optimizer.step()

        acc1, acc5 = accuracy(logits, labels, topk=(1, 5))
        losses.update(loss.item(), images[0].size(0))
        top1.update(acc1[0], images[0].size(0))
        batch_time.update(time.time() - end)
        end = time.time()

    progress.display(batch_idx)
    return top1.avg, losses.avg


def evaluate(model, eva_loader, ceriation, prefix, ignore=-1):
    losses = AverageMeter('Loss', ':3.2f')
    top1 = AverageMeter('Acc@1', ':3.2f')
    model.eval()

    with torch.no_grad():
        for i, (images, labels,_) in enumerate(eva_loader):
            images = Variable(images).cuda()
            labels = Variable(labels).cuda()
#             images = images.to(device)
#             labels = labels.to(device)

            logist = model(images)

            loss = ceriation(logist, labels)
            acc1, acc5 = accuracy(logist, labels, topk=(1, 5))

            losses.update(loss.item(), images[0].size(0))
            top1.update(acc1[0], images[0].size(0))

    if prefix != "":
        print(getTime(), prefix, round(top1.avg.item(), 2))

    return losses.avg, top1.avg.to("cpu", torch.float).item()

In [ ]:
best_acc = 0
for i in range(300):
    train_iter = iter(train_loader)
    print("Start training epoch:{}".format(i))
    train(model, train_iter, ceriation, optimizer,1)
    val_loss, val_acc = evaluate(model, val_loader, ceriation, "Epoch " + str(i) + ", Val Acc:")
    print(getTime(), "Model Best Val Acc:", val_acc)
    _, test_acc = evaluate(model, test_loader, ceriation, "Epoch_test "+ ", Test Acc1:")
    print(getTime(), "Model Best Test Acc:", test_acc)
    scheduler.step()    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(),'save_ori_model')